# Use Case 1: Defibrillator data from out-of-hospital cardiac arrest case

<div style = "font-size:20px">

This notebook illustrates the usage of the `vitabel` module to visualize, annotate and process time-series data from the medical field. Please find the detailed, searchable documentation here: 
[![Documentation Status](https://readthedocs.org/projects/vitabel/badge/?version=latest)](https://vitabel.readthedocs.io/en/latest/?badge=latest)

In this case we analyze data recorded by a defibrillator during a real-world out-of-hospital cardiac arrest case.
This notebook in particular demonstrates the capabilities of `vitabel` to **display** and **annote** data.

</div>

If you have never worked with _Jupyter Notebooks_ before, you may find this guide helpful: **[Beginners Guide to Jupyter Notebooks](https://mybinder.org/v2/gh/jupyter/notebook/HEAD?urlpath=%2Fdoc%2Ftree%2Fdocs%2Fsource%2Fexamples%2FNotebook%2FRunning+Code.ipynb)**

In [ ]:
from vitabel import Vitals, Label

import bz2
from pathlib import Path
import tempfile
import shutil
from datetime import datetime
import pandas as pd

## 1. Loading Data

<div style = "font-size:18px">

An empty vitabel object called `case` is initiated and all channels from a defibrillator recording (in this case a ZOLL X-Series case, stored in a compressed file) are loaded and added to the case.

</div>

In [ ]:
case = Vitals()

compressed_defi_file = Path("data/ZOLL_test_case.json.bz2")
with tempfile.TemporaryDirectory() as tmpdir:
    defi_file = Path(tmpdir) / compressed_defi_file.stem
    with bz2.open(compressed_defi_file, "rb") as source:
        with open(defi_file, "wb") as dest:
            shutil.copyfileobj(source, dest)

    case.add_defibrillator_recording(defi_file)

<div style = "font-size:18px">

Every vitable object as well as every channel and label holds a dictionary as `metadata`. In the following we add the usage of the case in this notebook to the metadata.

</div>

In [ ]:
usage_metadata = {
    "vitabel publication": {
        "user": "ENTER YOUR NAME HERE",
        "purpose": "highlighting capabilities to display and annotate defibrillator data",
        "remarks": [
            "a useful tool for researchers",
            "handling data of devices by other manufacturers should be integrated better in the future"
        ],
    }
}
case.metadata.update({"usage": usage_metadata})

print("Current case metadata:")
case.metadata

<div style = "font-size:18px">

We get an overview over all loaded channels by calling `case.info()`

</div>

In [ ]:
case.info()

## 2. Processing Data

<div style="font-size:18px">

We use integrated algorithms to extract **etCO₂** values and detect **ventilations** from the capnography signal.
Additionally, we analyze the accelerometer data from the CPR feedback device and the ECG signal from the defibrillation pads to estimate the **probability of spontaneous circulation**.

</div>

In [ ]:
case.compute_etco2_and_ventilations()
case.predict_circulation()

## 3. Adding new Labels

<div style = "font-size:18px">

We aim to manually **annotate** occurrences of **Return of Spontaneous Circulation (ROSC)**. To do so, we create an empty label and add it to the case. With `metadata` we can store miscellaneous data in the label inside a dictionary (`{}`). Furthermore, we can define the `plotstyle` by passing parameters in another dictionary.

The argument `plot_type` determines how label data is depicted in the plot. As the type is also dynamically adapted to the available data,
we can explore the different types by adding labels with different data in the later generated plot.

More details on labels can be found [in our documentation](https://vitabel.readthedocs.io/en/latest/autoapi/vitabel/timeseries/index.html#vitabel.timeseries.Label).

</div>

In [ ]:
ROSC_label = Label(
    name = "ROSC",
    time_index = [],
    data = [], 
    text_data = [], 
    metadata = {
        "source": "manual annotation",
        "label initialization time" : str(datetime.now())
    },
    plot_type = "combined",
    plotstyle = {"marker": "$\u2665$", "color": "red", "ms": 10, "linestyle": ""},   
)

case.add_global_label(ROSC_label)

<div style = "font-size:18px">

We see the newly added label (`ROSC`) by checking out all present labels in the case.

</div>

In [ ]:
case.get_label_names()

## 4. Plotting Data interactively

<div style = "font-size:18px">

We provide a setup for the interactive plot.

The `channels` to be plottted are defined as lists in a list (`[[],[]]`). Each of the inner lists define a new subplot.
The `labels` to be plotted on top are defined by another lists in a list.

Subplots below the already defined ones are defined by `channel_overviews`. These subplots serve a special purpose by displaying the entire recording interval and highlighting the depicted subsegement in the upper subplots.

Above the plot a menu is given to `Annotate`, `Align Timelines`, and deffines `Settings` of the plot.

Try to <u>annotate the ROSCs</u> by clicking with the right mouse button where you assume a ROSC.
Also try to toggle the checkboxes in various combinations in oder to <u>add different data</u> to the label.

</div>

In [ ]:
plot = case.plot_interactive(
    channels = [
        ["cpr_acceleration"], 
        ["capnography"], 
        ["ecg_pads"], 
        []
    ],
    labels = [
        ["ROSC", "cc_periods"],
        ["etco2_from_capnography", "ROSC"],
        ["ROSC"],
        ["ROSC", "rosc_probability"],
    ],
    channel_overviews = [["cpr_acceleration"]],
    time_unit = "s",
    subplots_kwargs = {"figsize": (16.5, 8)},
)
plot

<div style = "font-size:18px">

The data that has been added to the *ROSC* label can be retrieved by calling the `get_data` method. The result is returned in a `DataSlice` object.

</div>

In [ ]:
ROSC_label.get_data()

<div style = "font-size:18px">

By accessing `time_index`, `data`, or `text_data` we can explicitly obtain them from the output of `get_data`.

</div>

In [ ]:
dt = ROSC_label.get_data()
print(
    f"Time Index:     ", dt.time_index,
    f"\nNumeric Values: ", dt.data,
    f"\nTextual Values: ", dt.text_data,
    f"\n"
)

## 5. Storing Data

<div style="font-size:18px">

The entire case can be serialized and stored as a JSON file.

</div>


In [ ]:
case.save_data("case_1.json")

<div style = "font-size:18px">

Raw data of individual channels or labels can also be exported to a CSV file.

</div>

In [ ]:
case.get_label("ROSC").to_csv()
pd.read_csv("ROSC.csv", index_col = 0)

<div style="font-size:18px">

Alternatively, the label can also be serialized as a dictionary.

</div>

In [ ]:
case.get_label("ROSC").to_dict()